In [1]:
import pandas as pd

# URL del archivo Excel en formato raw
url = 'Apidae_INABIO_LIMPIA.xlsx'

# Leer el archivo Excel
df = pd.read_excel(url, engine='openpyxl', sheet_name=0)

# Listar las cabeceras (nombres de las columnas)
print("Cabeceras del archivo Excel:")
print(df.columns.tolist())  # Convierte los nombres de las columnas a una lista


# Reemplazar valores NaN por un valor por defecto (por ejemplo, una cadena vacía)
df = df.fillna('')  # Puedes reemplazar '' por cualquier otro valor como 0, 'Desconocido', etc.

# Reemplazar 'GalÃ¡pagos' por 'Galapagos' en todo el DataFrame
df = df.replace('GalÃ¡pagos', 'Galapagos', regex=True)

# Guardar el DataFrame limpio en un nuevo dataset llamado nuevo_df
nuevo_df = df

# Contar el total de registros (filas) después de las transformaciones
total_registros = nuevo_df.shape[0]

# Mostrar el total de registros
print(f"Total de registros en el DataFrame limpio: {total_registros}")


Cabeceras del archivo Excel:
['Familia', 'Genero', 'Especie', 'Pais', 'Provincia', 'Canton', 'Localidad', 'Latitud', 'Longitud', 'Altitud']
Total de registros en el DataFrame limpio: 1470


In [3]:
import pandas as pd

# Asumiendo que 'nuevo_df' ya está cargado

# Función para reemplazar los caracteres erróneos 'Ã­' y 'Ã±'
def limpiar_caracteres(texto):
    if isinstance(texto, str):
        # Reemplazar los caracteres erróneos por sus equivalentes correctos
        texto = texto.replace('Ã­', 'í').replace('Ã±', 'ñ')
    return texto

# Aplicar la función a todas las columnas de tipo texto en el DataFrame
nuevo_df = nuevo_df.applymap(limpiar_caracteres)

# Verificar los primeros registros del DataFrame limpio
print(nuevo_df.head())


  Familia  Genero Especie     Pais   Provincia               Canton  \
0  Apidae  Apidae          Ecuador  Tungurahua  Baños de Agua Santa   
1  Apidae  Apidae          Ecuador      Guayas            Guayaquil   
2  Apidae  Apidae          Ecuador      Guayas            Guayaquil   
3  Apidae  Apidae          Ecuador      El Oro               Torata   
4  Apidae  Apidae          Ecuador      El Oro               Torata   

           Localidad Latitud Longitud Altitud  
0     Casa del arbol  -1.418   -78.42  2600.0  
1  B.P. Cerro Blanco  -2.178  -80.016    20.0  
2  B.P. Cerro Blanco  -2.178  -80.016    20.0  
3            Sabayàn  -3.597  -79.846   370.0  
4            Sabayàn  -3.597  -79.846   370.0  


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_11504\3353647308.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  nuevo_df = nuevo_df.applymap(limpiar_caracteres)


In [5]:
import pandas as pd

# URL del archivo Excel en formato raw
url_ubicaciones = 'Ubicaciones.xlsx'

# Leer el archivo Excel
df_ubicaciones = pd.read_excel(url_ubicaciones, engine='openpyxl')

# Quitar espacios en blanco de todas las columnas
df_ubicaciones = df_ubicaciones.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Convertir todos los datos a minúsculas y poner la primera letra de cada palabra en mayúscula
df_ubicaciones = df_ubicaciones.apply(lambda x: x.str.title() if x.dtype == "object" else x)

# Mostrar las primeras filas del DataFrame
print(df_ubicaciones.head())


FileNotFoundError: [Errno 2] No such file or directory: 'Ubicaciones.xlsx'

In [300]:
import pandas as pd

# Supongamos que df_ubicaciones ya está definido
# Eliminar filas donde la Parroquia sea "Ecuador"
df_ubicaciones = df_ubicaciones.drop(df_ubicaciones[df_ubicaciones['Parroquia'] == "Ecuador"].index)

# Listar las cabeceras (nombres de las columnas)
print("Cabeceras del DataFrame:")
print(df_ubicaciones.columns.tolist())  # Convierte los nombres de las columnas a una lista



Cabeceras del DataFrame:
['Provincia', 'Canton', 'Parroquia']


In [302]:
from fuzzywuzzy import fuzz
import pandas as pd
import numpy as np

# Asumimos que 'nuevo_df' y 'df_ubicaciones' ya están cargados correctamente

# Creamos una lista de las provincias de df_ubicaciones (sin duplicados) en minúsculas
provincias_ubicaciones = df_ubicaciones['Provincia'].str.lower().unique()

# Definir una función para obtener la provincia más similar de df_ubicaciones usando fuzz.partial_ratio
def obtener_similitud_mejor(provincia_nueva, provincias_ubicaciones):
    # Usamos fuzz.partial_ratio para comparar la provincia completa con la provincia más corta
    similitudes = np.array([fuzz.partial_ratio(provincia_nueva, provincia) for provincia in provincias_ubicaciones])
    
    # Buscamos el índice del valor más alto que sea mayor al 70%
    max_similitud_idx = np.argmax(similitudes)
    if similitudes[max_similitud_idx] > 70:  # Ajustado a 70% de similitud
        return provincias_ubicaciones[max_similitud_idx]
    else:
        return None

# Aplicar la función a cada fila del DataFrame nuevo_df
nuevo_df['ProvinciaOK'] = nuevo_df['Provincia'].str.lower().apply(lambda x: obtener_similitud_mejor(x, provincias_ubicaciones))

# Mostrar el DataFrame con la columna 'ProvinciaOK' actualizada
print(nuevo_df.head())


  Familia     Genero      Especie     Pais   Provincia ProvinciaOK Canton  \
0  Apidae   Melitoma  segmentaria  Ecuador  Esmeraldas  esmeraldas          
1  Apidae   Melitoma  segmentaria  Ecuador  Esmeraldas  esmeraldas          
2  Apidae   Melitoma  segmentaria  Ecuador      Guayas      guayas          
3  Apidae  Epicharis      rustica  Ecuador  Esmeraldas  esmeraldas          
4  Apidae  Epicharis      rustica  Ecuador     Pastaza     pastaza          

  CantonOK  Localidad  Latitud  Longitud Altitud  
0           San Mateo  0.89169 -79.63306          
1           San Mateo  0.89169 -79.63306          
2           Guayaquil -2.20584 -79.90795          
3           San Mateo  0.89169 -79.63306          
4                Topo                             


In [304]:
# Guardar el nuevo dataframe en un archivo Excel
nuevo_df.to_excel('con_provinciaOK.xlsx', index=False)

In [305]:
from fuzzywuzzy import fuzz
import pandas as pd
import numpy as np

# Asumimos que 'nuevo_df' y 'df_ubicaciones' ya están cargados correctamente

# Creamos una lista de los cantones de df_ubicaciones (sin duplicados) en minúsculas
canton_ubicaciones = df_ubicaciones['Canton'].str.lower().unique()

# Definir una función para obtener el canton más similar de df_ubicaciones usando fuzz.partial_ratio
def obtener_similitud_mejor(localidad_nueva, canton_ubicaciones):
    # Usamos fuzz.partial_ratio para comparar la localidad completa con el canton más corto
    similitudes = np.array([fuzz.partial_ratio(localidad_nueva, canton) for canton in canton_ubicaciones])
    
    # Buscamos el índice del valor más alto que sea mayor al 70%
    max_similitud_idx = np.argmax(similitudes)
    if similitudes[max_similitud_idx] > 80:  # Ajustado a 70% de similitud
        return canton_ubicaciones[max_similitud_idx]
    else:
        return None

# Aplicar la función a cada fila del DataFrame nuevo_df
nuevo_df['CantonOK'] = nuevo_df['Canton'].str.lower().apply(lambda x: obtener_similitud_mejor(x, canton_ubicaciones))

# Mostrar el DataFrame con la columna 'CantonOK' actualizada
print(nuevo_df.head())


  Familia     Genero      Especie     Pais   Provincia ProvinciaOK Canton  \
0  Apidae   Melitoma  segmentaria  Ecuador  Esmeraldas  esmeraldas          
1  Apidae   Melitoma  segmentaria  Ecuador  Esmeraldas  esmeraldas          
2  Apidae   Melitoma  segmentaria  Ecuador      Guayas      guayas          
3  Apidae  Epicharis      rustica  Ecuador  Esmeraldas  esmeraldas          
4  Apidae  Epicharis      rustica  Ecuador     Pastaza     pastaza          

  CantonOK  Localidad  Latitud  Longitud Altitud  
0     None  San Mateo  0.89169 -79.63306          
1     None  San Mateo  0.89169 -79.63306          
2     None  Guayaquil -2.20584 -79.90795          
3     None  San Mateo  0.89169 -79.63306          
4     None       Topo                             


In [308]:
# Guardar el nuevo dataframe en un archivo Excel
nuevo_df.to_excel('con_ProvCanton.xlsx', index=False)

In [310]:
from fuzzywuzzy import fuzz
import pandas as pd
import numpy as np

# Asumimos que 'nuevo_df' y 'df_ubicaciones' ya están cargados correctamente

# Creamos una lista de los cantones de df_ubicaciones (sin duplicados) en minúsculas
canton_ubicaciones = df_ubicaciones['Canton'].str.lower().unique()

# Definir una función para obtener el canton más similar de df_ubicaciones usando fuzz.partial_ratio
def obtener_similitud_mejor(localidad_nueva, canton_ubicaciones):
    # Usamos fuzz.partial_ratio para comparar la localidad completa con el canton más corto
    similitudes = np.array([fuzz.partial_ratio(localidad_nueva, canton) for canton in canton_ubicaciones])
    
    # Buscamos el índice del valor más alto que sea mayor al 70%
    max_similitud_idx = np.argmax(similitudes)
    if similitudes[max_similitud_idx] > 85:  # Ajustado a 70% de similitud
        return canton_ubicaciones[max_similitud_idx]
    else:
        return None

# Función para realizar la búsqueda solo cuando la columna 'CantonOK' está vacía
def obtener_canton_si_necesario(row, canton_ubicaciones):
    # Si 'CantonOK' está vacío, buscamos en 'Localidad'
    if pd.isnull(row['CantonOK']) or row['CantonOK'] == '':
        canton_nueva = row['Localidad']  # Obtener la localidad
        canton_ok = obtener_similitud_mejor(canton_nueva, canton_ubicaciones)  # Buscar similitud
        return canton_ok
    else:
        # Si ya tiene valor en 'CantonOK', no hacer nada y devolver el valor original
        return row['CantonOK']

# Aplicar la función a cada fila del DataFrame nuevo_df
nuevo_df['CantonOK'] = nuevo_df.apply(lambda row: obtener_canton_si_necesario(row, canton_ubicaciones), axis=1)

# Mostrar el DataFrame con la columna 'CantonOK' actualizada
print(nuevo_df.head())



  Familia     Genero      Especie     Pais   Provincia ProvinciaOK Canton  \
0  Apidae   Melitoma  segmentaria  Ecuador  Esmeraldas  esmeraldas          
1  Apidae   Melitoma  segmentaria  Ecuador  Esmeraldas  esmeraldas          
2  Apidae   Melitoma  segmentaria  Ecuador      Guayas      guayas          
3  Apidae  Epicharis      rustica  Ecuador  Esmeraldas  esmeraldas          
4  Apidae  Epicharis      rustica  Ecuador     Pastaza     pastaza          

    CantonOK  Localidad  Latitud  Longitud Altitud  
0       None  San Mateo  0.89169 -79.63306          
1       None  San Mateo  0.89169 -79.63306          
2  guayaquil  Guayaquil -2.20584 -79.90795          
3       None  San Mateo  0.89169 -79.63306          
4       None       Topo                             


In [312]:
# Guardar el nuevo dataframe en un archivo Excel
nuevo_df.to_excel('con_ProvCantonOK.xlsx', index=False)

In [314]:
from fuzzywuzzy import fuzz
import pandas as pd
import numpy as np

# Asumimos que 'nuevo_df' y 'df_ubicaciones' ya están cargados correctamente

# Creamos una lista de las provincias de df_ubicaciones (sin duplicados) en minúsculas
provincias_ubicaciones = df_ubicaciones['Provincia'].str.lower().unique()

# Definir una función para obtener la provincia más similar de df_ubicaciones usando fuzz.partial_ratio
def obtener_similitud_mejor(provincia_nueva, provincias_ubicaciones):
    # Usamos fuzz.partial_ratio para comparar la provincia completa con la provincia más corta
    similitudes = np.array([fuzz.partial_ratio(provincia_nueva, provincia) for provincia in provincias_ubicaciones])
    
    # Buscamos el índice del valor más alto que sea mayor al 70%
    max_similitud_idx = np.argmax(similitudes)
    if similitudes[max_similitud_idx] > 70:  # Ajustado a 70% de similitud
        return provincias_ubicaciones[max_similitud_idx]
    else:
        return None

# Definir una función que haga la búsqueda en `Provincia` o `Localidad` según corresponda
def obtener_provincia_o_localidad(row, provincias_ubicaciones):
    # Si 'ProvinciaOK' está vacío, buscamos en 'Localidad'
    if pd.isnull(row['ProvinciaOK']) or row['ProvinciaOK'] == '':
        # Buscar en Localidad si ProvinciaOK está vacío
        localidad_nueva = row['Localidad'].lower()  # Convertir a minúsculas para comparación
        return obtener_similitud_mejor(localidad_nueva, provincias_ubicaciones)
    else:
        # Si ya tiene valor en 'ProvinciaOK', no hacer nada
        return row['ProvinciaOK']

# Aplicar la función a cada fila del DataFrame nuevo_df
nuevo_df['ProvinciaOK'] = nuevo_df.apply(lambda row: obtener_provincia_o_localidad(row, provincias_ubicaciones), axis=1)

# Mostrar el DataFrame con la columna 'ProvinciaOK' actualizada
print(nuevo_df.head())


  Familia     Genero      Especie     Pais   Provincia ProvinciaOK Canton  \
0  Apidae   Melitoma  segmentaria  Ecuador  Esmeraldas  esmeraldas          
1  Apidae   Melitoma  segmentaria  Ecuador  Esmeraldas  esmeraldas          
2  Apidae   Melitoma  segmentaria  Ecuador      Guayas      guayas          
3  Apidae  Epicharis      rustica  Ecuador  Esmeraldas  esmeraldas          
4  Apidae  Epicharis      rustica  Ecuador     Pastaza     pastaza          

    CantonOK  Localidad  Latitud  Longitud Altitud  
0       None  San Mateo  0.89169 -79.63306          
1       None  San Mateo  0.89169 -79.63306          
2  guayaquil  Guayaquil -2.20584 -79.90795          
3       None  San Mateo  0.89169 -79.63306          
4       None       Topo                             


In [316]:
# Guardar el nuevo dataframe en un archivo Excel
nuevo_df.to_excel('con_ProvCompletada.xlsx', index=False)

In [318]:
from fuzzywuzzy import fuzz
import pandas as pd
import numpy as np

# Asumimos que 'nuevo_df' y 'df_ubicaciones' ya están cargados correctamente

# Crear una lista de los cantones de df_ubicaciones (sin duplicados) en minúsculas
canton_ubicaciones = df_ubicaciones['Canton'].str.lower().unique()

# Definir una función para obtener el canton más similar de df_ubicaciones usando fuzz.partial_ratio
def obtener_similitud_mejor(localidad_nueva, canton_ubicaciones):
    # Usamos fuzz.partial_ratio para comparar la localidad completa con el canton más corto
    similitudes = np.array([fuzz.partial_ratio(localidad_nueva, canton) for canton in canton_ubicaciones])
    
    # Buscamos el índice del valor más alto que sea mayor al 70% de similitud
    max_similitud_idx = np.argmax(similitudes)
    if similitudes[max_similitud_idx] > 80:  # Ajustado a 70% de similitud
        return canton_ubicaciones[max_similitud_idx]
    else:
        return None

# Definir una función que haga la búsqueda en `Localidad` solo si `CantonOK` está vacío
def obtener_canton_si_necesario(row, canton_ubicaciones):
    # Si 'CantonOK' está vacío, buscamos en 'Localidad'
    if pd.isnull(row['CantonOK']) or row['CantonOK'] == '':
        # Buscar en Localidad si CantonOK está vacío
        localidad_nueva = row['Localidad'].lower()  # Convertir a minúsculas para comparación
        canton_ok = obtener_similitud_mejor(localidad_nueva, canton_ubicaciones)
        return canton_ok
    else:
        # Si ya tiene valor en 'CantonOK', no hacer nada y devolver el valor original
        return row['CantonOK']

# Aplicar la función a cada fila del DataFrame nuevo_df
nuevo_df['CantonOK'] = nuevo_df.apply(lambda row: obtener_canton_si_necesario(row, canton_ubicaciones), axis=1)

# Mostrar el DataFrame con la columna 'CantonOK' actualizada
print(nuevo_df.head())


  Familia     Genero      Especie     Pais   Provincia ProvinciaOK Canton  \
0  Apidae   Melitoma  segmentaria  Ecuador  Esmeraldas  esmeraldas          
1  Apidae   Melitoma  segmentaria  Ecuador  Esmeraldas  esmeraldas          
2  Apidae   Melitoma  segmentaria  Ecuador      Guayas      guayas          
3  Apidae  Epicharis      rustica  Ecuador  Esmeraldas  esmeraldas          
4  Apidae  Epicharis      rustica  Ecuador     Pastaza     pastaza          

    CantonOK  Localidad  Latitud  Longitud Altitud  
0       None  San Mateo  0.89169 -79.63306          
1       None  San Mateo  0.89169 -79.63306          
2  guayaquil  Guayaquil -2.20584 -79.90795          
3       None  San Mateo  0.89169 -79.63306          
4       None       Topo                             


In [320]:
# Guardar el nuevo dataframe en un archivo Excel
nuevo_df.to_excel('con_CantonCompletado.xlsx', index=False)